In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats, anisotropy
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



## Load and concatenate pixel strains and harmonic coefficients

In [ ]:
file1 = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss18_n100_r40_d15_f30_l1000_p0/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r40_b00-49.npz')
print(f"{file1.files=}")
# load ss info
shape = file1['ss_shape']
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
fobs = file1['fobs']

# load ranking info
nsort = file1['nsort']
fidx = file1['fidx']
hc_ref = file1['hc_ref']

# load harmonics info
nside = file1['nside']
lmax  = file1['lmax']

# load map and harmonics from 1st split chunk
moll_hc_best1 = file1['moll_hc_best']
Cl_best1 = file1['Cl_best']
nbest1 = len(moll_hc_best1)

file1.close()

# load map and harmonics from 2nd split chunk
file2 = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss18_n100_r40_d15_f30_l1000_p0/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r40_b50-99.npz')
moll_hc_best2 = file2['moll_hc_best']
Cl_best2 = file2['Cl_best']
nbest2 = len(moll_hc_best2)
file2.close()


In [ ]:

# concatenate Cl and moll_hc
nbest=nbest1+nbest2
print(nbest)
npix = hp.nside2npix(nside)
Cl_best = np.concatenate((Cl_best1, Cl_best2)).reshape(nbest, nfreqs, nreals, lmax+1)
moll_hc_best = np.concatenate((moll_hc_best1, moll_hc_best2), axis=0).reshape(nbest, nfreqs, nreals, npix)



In [ ]:
print(f"{shape=}, {nsort.shape=}, {nbest=}")
print(f"{Cl_best1.shape=}, {moll_hc_best1.shape=}")
print(f"{Cl_best2.shape=}, {moll_hc_best2.shape=}")
print(f"{Cl_best.shape=}, {moll_hc_best.shape=}")

In [ ]:
# npix = anisotropy.NPIX
# lmax = anisotropy.LMAX

# Cl_best = np.zeros((nbest, nfreqs, nreals, lmax+1 ))
# moll_hc_best = np.zeros((nbest, nfreqs, nreals, npix))
# for nn in range(nbest):
#     print('on nn=%d out of nbest=%d' % (nn,nbest))
#     moll_hc_best[nn,...], Cl_best[nn,...] = anisotropy.sph_harm_from_hc(hc_ss[nsort[nn]], 
#                                             hc_bg[nsort[nn]])

In [ ]:
print(np.where(Cl_best[:,0,0,0]==0))
print(np.where(Cl_best1[:,0,0,0]==0))
print(np.where(Cl_best2[:,0,0,0]==0))

In [ ]:
plt.scatter(np.arange(len(Cl_best)), Cl_best[:,0,0,0])
plt.ylim(0,0.1e-58)

In [ ]:

print(holo.utils.stats(Cl_best1))
print(holo.utils.stats(Cl_best2))

In [ ]:
print(Cl_best.shape)
print(holo.utils.stats(Cl_best))

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
def plot_ClC0_medians(xx, Cl_best, lmax, nshow):
    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=-1) # (B,F,l) median over realizations

    colors = cm.gist_rainbow(np.linspace(0, 1, lmax))
    for ll in range(lmax):
        ax.plot(xx, np.median(yy[:,:,ll], axis=0), color=colors[ll], label='$l=%d$' % (ll+1))
        for pp in [50, 98]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,:,ll], percs, axis=0), alpha=0.1, color=colors[ll])
        
        for bb in range(0,nshow):
            ax.plot(xx, yy[bb,:,ll], color=colors[ll], linestyle=':', alpha=0.1,
                                 linewidth=1)         
        ax.legend(ncols=2)
        plot._twin_hz(ax, nano=False)
    
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
fig = plot_ClC0_medians(fobs_yrs, Cl_best, lmax, nshow=10)
fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
# def plot_Clg0_llp1_medians(xx, Cl_best, lmax, nshow):
#     fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

#     yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)
#     yy = np.median(yy, axis=-1) # (B,F) median over realizations

#     ax.plot(xx, np.median(yy, axis=0), color='k', label='median of samples, $l_\mathrm{max}=%d$' % lmax)
#     for pp in [50, 98]:
#         percs = pp/2
#         percs = [50-percs, 50+percs]
#         ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.1, color='k')

#     colors = cm.rainbow(np.linspace(1, 0, nshow))
#     bb=0
#     ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.4,
#                 linewidth=1, label="individual best samples, median of realizations")
#     for bb in range(1,nshow):
#         ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.4,
#                 linewidth=1)
#     ax.legend()
#     plot._twin_hz(ax, nano=False)
    
#     # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
#     #             % (nbest, lmax))
#     return fig
# fig = plot_Clg0_llp1_medians(fobs_yrs, Cl_best, lmax, nshow=50)

# Add Nihan's Data

In [ ]:
# decision threshold from the frequentist analysis for l = 0 through l = 6
freq_dthresh = np.array([1.        , 0.39622564, 0.32362566, 0.20984702, 0.11960839,
       0.07806638, 0.04426357])

# This is for the all-band Bayesian analysis from l = 1 to l = 6:
bayes_Cl = np.array([0.24187743, 0.17480158, 0.10009671, 0.08109598, 0.05493891,
       0.0317599 ])

# And these are the per-frequency Bayesian upper limits, 
# where each row represents the frequency bin from low to high (0 to 4) 
# and then each column is the spherical harmonic multipole from l = 1 to l = 6:
# (F, l)
Cl_nihan = np.array([
    [0.20216773, 0.14690035, 0.09676646, 0.07453352, 0.05500382, 0.03177427],
    [0.21201336, 0.14884939, 0.10545698, 0.07734305, 0.05257189, 0.03090662],
    [0.20840993, 0.14836757, 0.09854803, 0.07205384, 0.05409881, 0.03305785],
    [0.19788951, 0.15765126, 0.09615489, 0.07475364, 0.0527356 , 0.03113331],
    [0.20182648, 0.14745265, 0.09681202, 0.0746824 , 0.05503161, 0.0317012 ]])
print(Cl_nihan.shape)

freq_bins_Hz = np.array([2.0, 4.0, 5.9, 7.9, 9.9]) *10**-9 # Hz

In [ ]:
print(Cl_nihan[:,0])

In [ ]:
def plot_ClC0_medians(xx, Cl_best, lmax, nshow, xx_Nihan, Cl_nihan):
    fig, ax = plot.figax(figsize=(7,6), xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell}/C_0$')

    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=-1) # (B,F,l) median over realizations

    # colors = ['k', 'b', 'r', 'g', 'c', 'm']
    colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']
    for ll in range(lmax):
        ax.plot(xx, np.median(yy[:,:,ll], axis=0), color=colors[ll]) #, label='median of samples, $l=%d$' % ll)
        for pp in [50, 95]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,:,ll], percs, axis=0), alpha=0.1, color=colors[ll])
        
        for bb in range(0,nshow):
            # if ll==0 and bb==0:
            #     label = "individual best samples, median of realizations"
            # else: 
            label=None
            ax.plot(xx, yy[bb,:,ll], color=colors[ll], linestyle=':', alpha=0.25,
                                 linewidth=1, label=label)
            
        # Add Nihan's data
        ax.plot(xx_Nihan, Cl_nihan[:,ll], 
                label = '$l=%d$' % (ll+1), 
            color=colors[ll], marker='o', ms=8)
        
        plot._twin_yr(ax, nano=False)
    
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
# fig1 = plot_ClC0_medians(fobs_yrs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz * YR, Cl_nihan=Cl_nihan)


In [ ]:
print(Cl_best.shape)

In [ ]:
print('%e' % (1/YR))

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan)
ax = fig.axes[0]

# Sato-Polito & Kamionkowski
spk_xx = np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7])
spk_yy = np.array([1*10**-5, 1*10**-3, 1*10**-1])

# ax.plot(spk_xx * YR, spk_yy, label='SP & K', color='tab:orange')
ax.set_xlim(fobs[0]-10**-10, 1/YR)

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
fig.text(0.05,1, 'nside=%d, mbp-ss18, l=1000, uniform02B, 100 best out of 100 samples' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best[:10], lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan)
ax = fig.axes[0]

# Sato-Polito & Kamionkowski
spk_xx = np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7])
spk_yy = np.array([1*10**-5, 1*10**-3, 1*10**-1])

# ax.plot(spk_xx * YR, spk_yy, label='SP & K', color='tab:orange')
ax.set_xlim(fobs[0]-10**-10, 1/YR)

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3)

fig.text(0.05,1, 'nside=%d, mbp-ss18, l=1000, uniform02B, 10 best out of 100 samples' % nside)
fig.tight_layout()

### Only 1 realization harmonic

In [ ]:
def plot_C1C0_medians(xx, Cl_best, lmax, lreals, nshow, xx_Nihan, Cl_nihan):
    fig, ax = plot.figax(figsize=(7,6), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell}/C_0$')

    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=-1) # (B,F,l) median over realizations

    # colors = ['k', 'b', 'r', 'g', 'c', 'm']
    colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']
    
    for ll in lreals:
        ax.plot(xx, np.median(yy[:,:,ll], axis=0), color=colors[ll]) #, label='median of samples, $l=%d$' % ll)
        for pp in [50, 95]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,:,ll], percs, axis=0), alpha=0.1, color=colors[ll])

        for bb in range(0,nshow):
            # if ll==0 and bb==0:
            #     label = "individual best samples, median of realizations"
            # else: 
            label=None
            ax.plot(xx, yy[bb,:,ll], color=colors[ll], linestyle=':', alpha=0.25,
                                linewidth=1, label=label)
            
        # Add Nihan's data
    for ll in range(lmax):
        ax.plot(xx_Nihan, Cl_nihan[:,ll], 
                label = '$l=%d$' % (ll+1), 
            color=colors[ll], marker='o', ms=8)
        
        plot._twin_hz(ax, nano=False)
    
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
# fig1 = plot_ClC0_medians(fobs_yrs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz * YR, Cl_nihan=Cl_nihan)


In [ ]:
fig = plot_C1C0_medians(fobs_yrs, Cl_best, lmax=6, lreals=[0,], nshow=10, xx_Nihan=freq_bins_Hz * YR, Cl_nihan=Cl_nihan)
ax = fig.axes[0]

# Sato-Polito & Kamionkowski
spk_xx = np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7])
spk_yy = np.array([1*10**-5, 1*10**-3, 1*10**-1])

# ax.plot(spk_xx * YR, spk_yy, label='SP & K', color='tab:orange')
ax.set_xlim(fobs[0]*YR-.005, 1)

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3)

fig.tight_layout()
fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/model07a_ClC0_1ell.png', dpi=300)

In [ ]:
fig = plot_C1C0_medians(fobs_yrs, Cl_best, lmax=6, lreals=[0,5], nshow=10, xx_Nihan=freq_bins_Hz * YR, Cl_nihan=Cl_nihan)
ax = fig.axes[0]

# Sato-Polito & Kamionkowski
spk_xx = np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7])
spk_yy = np.array([1*10**-5, 1*10**-3, 1*10**-1])

# ax.plot(spk_xx * YR, spk_yy, label='SP & K', color='tab:orange')
ax.set_xlim(fobs[0]*YR-.005, 1)

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3)

fig.tight_layout()
fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/model07a_ClC0_2ell.png', dpi=300)

In [ ]:
# fig = plot_ClC0_medians(fobs_yrs, Cl_best, lmax, nshow=100)
# ax = fig.axes[0]

# xx = freq_bins_Hz * YR 
# for ll in range(len(Cl_nihan[0])):
    

# ax.set_xlim(0.18*10**-8*YR, 1.2*10**-8*YR)
# # ax.set_xscale('linear')
# fig.legend(loc='lower right')

In [ ]:
# fig = plot_Clg0_llp1_medians(fobs_yrs, Cl_best, lmax, nshow=50)
# ax = fig.axes[0]

# xx = freq_bins_Hz * YR 
# ax.plot(xx, Clg0_nihan, label = 'per-frequency Bayesian upper limits', color='tab:blue', marker='o', ms=8)

# # ax.set_xlim(0.18*10**-8*YR, 1.2*10**-8*YR)
# ax.legend(loc='lower right')

# Add Sato-Polito & Kamionkowski

In [ ]:
fig = plot_Clg0_llp1_medians(fobs_yrs, Cl_best, lmax, nshow=50)
ax = fig.axes[0]

xx = freq_bins_Hz * YR 
ax.plot(xx, Clg0_nihan, label = 'per-frequency Bayesian upper limits', color='tab:blue', marker='o', ms=8)

# Sato-Polito & Kamionkowski
spk_xx = np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7])
spk_yy = np.array([1*10**-5, 1*10**-3, 1*10**-1])

ax.plot(spk_xx * YR, spk_yy, label='SP & K Rough Estimate', color='tab:orange')
ax.set_xlim(0, np.max(fobs_yrs))
ax.legend()
fig

In [ ]:
print(nsort)

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000'

hdfname = sspath+'/sam_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
print('Best sample:', nsort[0])
hc_ss = ssfile['hc_ss'][nsort[0],...]
hc_bg = ssfile['hc_bg'][nsort[0],...]
fobs = ssfile['fobs'][:]
ssfile.close()

shape = hc_ss.shape
nfreqs, nreals, nloudest = shape[0], shape[1], shape[2]
print('F,R,L =', nfreqs, nreals, nloudest)


# Scratch

In [ ]:
split=7
nbest=100
for ss in range(split):
    bestrange = (np.array([ss, (ss+1)])*(nbest)/split).astype(int)
    bestrange[1] = np.min([bestrange[1], nbest])
    print(f"{bestrange=}")

In [ ]:
for ii, nn in enumerate(range(bestrange[0], bestrange[1])):
            print('on nn=%d out of nbest=%d, index=%d' % (nn,nbest, ii))